# Project Notebook: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [ ]:
# Importing pandas
import pandas as pd
import numpy as np

pd.options.display.max_columns = 99

# Your code goes here
personal_loan_df = pd.read_csv('https://bit.ly/3H2XVgC')
personal_loan_df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [ ]:
personal_loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          42538 non-null  object 
 1   member_id                   42535 non-null  float64
 2   loan_amnt                   42535 non-null  float64
 3   funded_amnt                 42535 non-null  float64
 4   funded_amnt_inv             42535 non-null  float64
 5   term                        42535 non-null  object 
 6   int_rate                    42535 non-null  object 
 7   installment                 42535 non-null  float64
 8   grade                       42535 non-null  object 
 9   sub_grade                   42535 non-null  object 
 10  emp_title                   39909 non-null  object 
 11  emp_length                  41423 non-null  object 
 12  home_ownership              42535 non-null  object 
 13  annual_inc                  425

In [ ]:
# calculating the memory usage for the first 1000 rows
personal_loan_df1000 = pd.read_csv('https://bit.ly/3H2XVgC', nrows= 1000)
print('Memory usage(mb)for 1000 rows:', personal_loan_df1000.memory_usage(deep=True).sum()/(1024^2))

Memory usage(mb)for 1000 rows: 1560.9746588693956


In [ ]:
# we increase the number of rows 4000 to find out if the memory usage > 5mb.
personal_loan_df= pd.read_csv("https://bit.ly/3H2XVgC", nrows=4000)
personal_loan_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          4000 non-null   int64  
 1   member_id                   4000 non-null   float64
 2   loan_amnt                   4000 non-null   float64
 3   funded_amnt                 4000 non-null   float64
 4   funded_amnt_inv             4000 non-null   float64
 5   term                        4000 non-null   object 
 6   int_rate                    4000 non-null   object 
 7   installment                 4000 non-null   float64
 8   grade                       4000 non-null   object 
 9   sub_grade                   4000 non-null   object 
 10  emp_title                   3751 non-null   object 
 11  emp_length                  3873 non-null   object 
 12  home_ownership              4000 non-null   object 
 13  annual_inc                  4000 

**Finding**: 
memory usage is proportional to the number of rows processed

## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

In [ ]:
# finding the columns that have numeric types
numeric_type = pd.read_csv("https://bit.ly/3H2XVgC", chunksize=1000)
numeric_list  = list()
for x in numeric_type:
  numeric_list.append(len((x.select_dtypes(include=np.number)).columns))

print(numeric_list)

[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 31, 31, 30]


In [ ]:
# finding columns with string values
string_type = pd.read_csv("https://bit.ly/3H2XVgC", chunksize=1000)
string_list = list()
for x in string_type:
  string_list.append(len((x.select_dtypes(include='object')).columns))

print(string_list)

[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 21, 21, 22]


In [ ]:
# fing unique values in each string column and finding number of string columns with values < 50 % unique

# using chunks of 2000 rows to read data
chunks_unique = pd.read_csv("https://bit.ly/3H2XVgC", chunksize=3000)

numeric_unique_values = []
numeric_cols_percent_unique = []
# loading each chunk
for x in chunks_unique:
    x['id'] = pd.to_numeric(x['id'], errors='coerce')
    x = x.dropna(axis=0, subset=['id']) 
    # picking only the string columns
    string_cols = x.select_dtypes(include='object')

    # finding the number of unique values in each string column
    numeric_unique_values.append(string_cols.nunique())
    numeric_unique = pd.concat(numeric_unique_values)
    numeric_unique = numeric_unique.groupby(numeric_unique.index).sum()
  
    numeric_cols_percent_unique.append(string_cols.nunique()/string_cols.count() *100)
    numeric_cols_percent = pd.concat(numeric_cols_percent_unique)
    numeric_cols_percent = numeric_cols_percent.groupby(numeric_cols_percent.index).mean()

    num_cols_less_than_50_percent_unique = list((numeric_cols_percent[numeric_cols_percent <50]).index)
print(numeric_unique)
print(numeric_cols_percent)
print(num_cols_less_than_50_percent_unique)

addr_state               653
application_type          15
earliest_cr_line        5537
emp_length               165
emp_title              36958
grade                    105
home_ownership            52
initial_list_status       15
int_rate                1124
issue_d                  113
last_credit_pull_d      1078
last_pymnt_d             911
loan_status               58
purpose                  201
pymnt_plan                16
revol_util             13590
sub_grade                522
term                      27
title                  26923
verification_status       43
zip_code                8117
dtype: int64
addr_state              1.880203
application_type        0.043550
earliest_cr_line       14.686817
emp_length              0.488414
emp_title              92.775698
grade                   0.304848
home_ownership          0.156422
initial_list_status     0.043550
int_rate                3.376513
issue_d                 0.343083
last_credit_pull_d      3.309096
last_pymnt_d   

In [ ]:
#finding clumns with float data types and having no missing values 
chunks_float= pd.read_csv("https://bit.ly/3H2XVgC", chunksize=2000)
float_cols_no_missing_values_list = []
for x in chunks_float:
  float_cols = x.select_dtypes(include='float')
  missing_values = float_cols.isnull().sum()
  float_cols_no_missing_values = [col for col in float_cols if missing_values[col] == 0]
  float_cols_no_missing_values_list.append(float_cols_no_missing_values)
  float_cols_no_missing_values_series = pd.concat([pd.Series(cols) for cols in float_cols_no_missing_values_list])
print(float_cols_no_missing_values_series)

0                    member_id
1                    loan_amnt
2                  funded_amnt
3              funded_amnt_inv
4                  installment
                ...           
24                 policy_code
25              acc_now_delinq
26    chargeoff_within_12_mths
27                 delinq_amnt
28                   tax_liens
Length: 598, dtype: object


<ipython-input-18-b1e011777419>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  float_cols_no_missing_values_series = pd.concat([pd.Series(cols) for cols in float_cols_no_missing_values_list])


In [ ]:
# finding total memory usage
memory_usage = pd.read_csv("https://bit.ly/3H2XVgC", chunksize=3000)
total_memory_usage = 0
for x in memory_usage:
  chunk_memory_usage = x.memory_usage(deep=True).sum()
  total_memory_usage += chunk_memory_usage
print(f'Total memory in use is: {total_memory_usage/2**20} MB')

Total memory in use is: 65.24251079559326 MB


## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [ ]:
# converting columns to categorical data as follows
categ_collumns_conversion= {"sub_grade": "category", "purpose": "category", "home_ownership": "category", 
    "verification_status": "category"}

categ_col_chunks = pd.read_csv("https://bit.ly/3H2XVgC",
                               chunksize=3000, 
                               parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"],
                     dtype = categ_collumns_conversion)

total_memory_usage = 0

for x in categ_col_chunks:
    x['id'] = pd.to_numeric(x['id'], errors='coerce')
    x = x.dropna(axis=0, subset=['id']) 
    
    string_cols = x.select_dtypes(include='object')
    
    x["term"] = pd.to_numeric(x["term"].str.replace(" ", "").str.replace("months", "")) 
    x["int_rate"] = pd.to_numeric(x["int_rate"].str.replace("%", ""))
    x["revol_util"]= pd.to_numeric(x["revol_util"].str.replace("%", ""))

    
    chunk_memory_usage = x.memory_usage(deep=True).sum()

    
    total_memory_usage += chunk_memory_usage

print(f'Total memory in use is: {total_memory_usage/2**20} MB')



Total memory in use is: 38.948533058166504 MB


**finding:**
its clear that the memory usage here of 39mb is a bit lower compared with the previous usage which was 65mb with the same chunk size of 3000.



## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




In [ ]:
# Your code goes here
chunks_optimal = pd.read_csv("https://bit.ly/3H2XVgC",
                     chunksize=2000, 
                     parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"],
                     dtype = categ_collumns_conversion)

total_memory_usage = 0

for x in chunks_optimal:
    x['id'] = pd.to_numeric(x['id'], errors='coerce')
    x = x.dropna(axis=0, subset=['id']) 
    
    x["term"] = pd.to_numeric(x["term"].str.replace(" ", "").str.replace("months", "")) 
    x["int_rate"] = pd.to_numeric(x["int_rate"].str.replace("%", ""))
    x["revol_util"]= pd.to_numeric(x["revol_util"].str.replace("%", ""))

    missing_values = float_cols.isnull().sum()
    float_collumns = x.select_dtypes(include='float')
    float_columns_missing_values = [col for col in float_cols if missing_values[col] > 0]
    float_columns_no_missing_values = [col for col in float_cols if missing_values[col] == 0]
    for col in float_columns_missing_values:
        x[col] = pd.to_numeric(x[col], downcast='float')


    for col in float_cols_no_missing_values:
        x[col] = pd.to_numeric(x[col], downcast='integer')

    memory_usage = x.memory_usage(deep=True).sum()
    total_memory_usage += memory_usage

print(f'Memory in use is: {total_memory_usage/2**20} MB')


Memory in use is: 34.11496448516846 MB


**Finding:**
its found out that memory in use reduces further, indicating that more memory is saved when we optimize the numeric columns using pandas.to_numeric() function

## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [ ]:
# Your code goes here
#create a function to determine the optimal chunk size
def chunk_size(file_path, memory_limit):

#check two rows of the data set
  optimal_df = pd.read_csv(file_path, nrows=2)

#finding memory usage of two rows
  row_memory_usage = optimal_df.memory_usage().sum()

#finding maximum number of rows that can be processed per time

  maximum_rows = memory_limit / row_memory_usage
  chunk_size = int(maximum_rows)
    
  return chunk_size
#print('chunk_size')

In [ ]:
#create a function to determine the the string columns convertible to numeric by removing %

def string_columns_convertible(df):
    
    # Select all the string columns
    string_cols = optimal_df.select_dtypes(include='object')
    
    string_cols_to_convert = []
    
    # #looping through the string columns
    for col in string_cols:
        try:
            pd.to_numeric(string_cols[col].str.replace('%', ''), errors='coerce')
            string_cols_to_convert.append(col)
        except:
            pass
    
    return string_cols_to_convert

In [ ]:
# Determine which numeric columns can be converted to more space efficient representations.
def numeric_columns_convertible(optimal_df):
  numeric_columns = optimal_df.select_dtypes(include=['int', 'float'])
  numeric_columns_missing_values = []
  numeric_columns_no_missing_values = []

  for columns in numeric_columns:
        missing_values = numeric_columns[col].isnull().sum()
        if missing_values > 0:
            numeric_columns_missing_values.append(col)
        else:
            numeric_columns_no_missing_values.append(col)
    
  return numeric_columns_missing_values, numeric_columns_no_missing_values
